Libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import plotly.graph_objects as po
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.contrib import appsi
import os

Inputs

In [2]:
# date to start computing from
date_start = dt.date(2023,11,1)

# MND storage
mnd_max_wr = 917
mnd_wgv_cap = 55000
mnd0 = mnd_wgv_cap

# current working gas volume, injection rate and withdrawal rate
wgv1 = 2133317 - mnd_wgv_cap
ir1 = 23649
wr1 = 31863

# initial state of gas storage
z0 = wgv1

# date of changing wgv, ir, wr
date_change = dt.date(2024, 4, 1)

# z = 0 between seasons
empty_storage = False
empty_on = dt.date(2024,4,1)

# changed wgv, ir, wr values
wgv2 = 1683880
ir2 = 18577
wr2 = 25184

# terminal date
date_end = dt.date(2025, 3, 31)

# injection season months
season_inj = [4, 9]

# prices from excel (TTF)
prices_monthly = pd.read_excel("prices.xlsx", parse_dates=['date'], usecols=['date', 'price'])

# injection and withdrawal restrictions
inj_dict = np.array([[0,8,16,48,75,82,91,96],
                     [8,16,48,75,82,91,96,100],
                     [50,75,100,93,80,65,55,20]])/100
wit_dict = np.array([[0,3,15,35,72,85],
                     [3,15,35,72,85,100],
                     [18,53,78,100,73,40]])/100

# minimal states of gas storage in first days of given months
state_to_date = {5: 0.05, 7: 0.3, 9: 0.6, 11: 0.9}

# for bsd, we use different curve
bsd_wit_dict = np.array([[0,3,15,35,72,85],
                         [3,15,35,72,85,100],
                         [18,53,78,100,100,100]])/100

# BSD SY 23/24
check_bsd = False
bsd_wr = {dt.date(2023,10,1):15099.8,dt.date(2023,11,1):26424.7,dt.date(2023,12,1):33974.6,dt.date(2024,1,1):37749.6,dt.date(2024,2,1):33974.6,dt.date(2024,3,1):26424.7}
bsd_wgv = {dt.date(2023,10,1):379337.1,dt.date(2023,11,1):663839.9,dt.date(2023,12,1):853508.5,dt.date(2024,1,1):948342.7,dt.date(2024,2,1):853508.5,dt.date(2024,3,1):663839.9}

Data prep

In [3]:
delta = dt.timedelta(days=1)
dates = [date_start + dt.timedelta(days=i) for i in range(0,(date_end-date_start).days+1)]

prices_monthly = prices_monthly[prices_monthly['date'] >= pd.Timestamp(date_start.replace(day=1))]
prices_monthly = prices_monthly.reset_index(drop=True)
prices_monthly['year'] = prices_monthly['date'].dt.year
prices_monthly['month'] = prices_monthly['date'].dt.month
prices_monthly = prices_monthly.drop('date', axis=1)

prices_daily = pd.DataFrame({'date': pd.to_datetime(dates)})
prices_daily['year'] = prices_daily['date'].dt.year
prices_daily['month'] = prices_daily['date'].dt.month
prices_daily['day'] = prices_daily['date'].dt.day
prices_daily = pd.merge(prices_monthly, prices_daily, on=['year', 'month'])
prices_daily['date_datetime'] = prices_daily['date'].dt.date
prices_daily = prices_daily.rename(columns={'date': 'date_timestamp'})
prices_daily = prices_daily[['date_timestamp', 'date_datetime', 'year', 'month', 'day', 'price']]

MND gas storage

In [4]:
prices_monthly_select = prices_monthly[
    ((prices_monthly['month'] < season_inj[0]) & (prices_monthly['year'] == 2024)) | 
    ((prices_monthly['month'] > season_inj[1]) & (prices_monthly['year'] == 2023))
]
prices_monthly_sort = prices_monthly_select.sort_values('price', ascending=False)

mnd_wr = {key: 0 for key in dates}
mnd_state = {key: 0 for key in dates}

months_to_wit = []
par = mnd_wgv_cap
for i in prices_monthly_sort[['month','year']].itertuples():
    start = dt.date(i.year,i.month,1)
    if i.month == 12:
        end = dt.date(i.year+1,1,1)
    else:
        end = dt.date(i.year,i.month+1,1)
    for j in range(0, (end-start).days):
        if par <= mnd_max_wr:
            mnd_wr_actual = par
        else:
            mnd_wr_actual = mnd_max_wr
        par -= mnd_wr_actual
        if par == 0:
            break_statement = True
            break
        else:
            break_statement = False
    months_to_wit.append(i)
    months_to_wit.sort()
    if break_statement:
        break

months_to_wit_tup = []
for i in months_to_wit:
    months_to_wit_tup.append((i.month,i.year))

empty = False
for day in dates:
    if day == date_start:
        mnd_state[day] = mnd0
        continue
    mnd_wr_actual = 0
    if ((day.month, day.year) in months_to_wit_tup) and not empty:
        if mnd_state[day-delta] <= mnd_max_wr:
            mnd_wr_actual = mnd_state[day-delta]
            empty == True
        else:
            mnd_wr_actual = mnd_max_wr
    mnd_state[day] = mnd_state[day-delta] - mnd_wr_actual
    mnd_wr[day] = mnd_wr_actual

mnd_state_ar = np.array(list(mnd_state.values()))
mnd_wr_ar = -np.array(list(mnd_wr.values()))

Model

In [5]:
# def model():
# -----------------------------------------------------------
# Model

m = pyo.ConcreteModel(name='OptimusGas')


# -----------------------------------------------------------
# Sets

injection_idx = [1, 2, 3, 4, 5, 6, 7, 8]
withdrawal_idx = [1, 2, 3, 4, 5, 6]
tab_val = ['lower', 'upper', 'portion']
bsd_months = [list(bsd_wgv.keys())[i].month for i in range(0,len(bsd_wgv))]

m.i = pyo.Set(initialize=dates)
m.j = pyo.Set(initialize=injection_idx)
m.k = pyo.Set(initialize=withdrawal_idx)
m.tab_val = pyo.Set(initialize=tab_val)
m.months = pyo.Set(initialize=list(state_to_date.keys()))
m.bsd_months = pyo.Set(initialize=bsd_months)

n_inj = len(injection_idx)
n_wit = len(withdrawal_idx)
n = len(dates)


# -----------------------------------------------------------
# Parameters

price = prices_daily[['date_datetime', 'price']].set_index('date_datetime').T.to_dict('records')[0]

wgv = {date_start: wgv1, date_change: wgv2}
ir = {date_start: ir1, date_change: ir2}
wr = {date_start: wr1, date_change: wr2}
m_const = {date_start: wgv[date_start] + 100000, date_change: wgv[date_change] + 100000}

wgv_dict = {dates[i]: wgv[date_start] if dates[i] < date_change else wgv[date_change] for i in range(n)}
ir_dict = {dates[i]: ir[date_start] if dates[i] < date_change else ir[date_change] for i in range(n)}
wr_dict = {dates[i]: wr[date_start] if dates[i] < date_change else wr[date_change] for i in range(n)}
m_const_dict = {dates[i]: m_const[date_start] if dates[i] < date_change else m_const[date_change] for i in range(n)}

inj_dict_days={}
wit_dict_days={}
bsd_wit_dict_days={}
for j in range(n_inj):
    for l in range(3):
        inj_dict_days[injection_idx[j],tab_val[l]] = inj_dict[l,j]
for j in range(n_wit):
    for l in range(3):
        wit_dict_days[withdrawal_idx[j],tab_val[l]] = wit_dict[l,j]
        bsd_wit_dict_days[withdrawal_idx[j],tab_val[l]] = bsd_wit_dict[l,j]            

bsd_wr_days = {}
for i in dates:
    bsd_wr_days[i] = 0
    for d in bsd_wr.keys():
        if d <= i and d.month == i.month and d.year == i.year:
            bsd_wr_days[i] = bsd_wr[d]
bsd_wgv_days = {}
for i in dates:
    bsd_wgv_days[i] = 0
    for d in bsd_wgv.keys():
        if d <= i and d.month == i.month and d.year == i.year:
            bsd_wgv_days[i] = bsd_wgv[d]

m.p = pyo.Param(m.i, initialize=price)
m.wgv = pyo.Param(m.i, initialize=wgv_dict)
m.ir = pyo.Param(m.i, initialize=ir_dict)
m.wr = pyo.Param(m.i, initialize=wr_dict)
m.m_const = pyo.Param(m.i, initialize=m_const_dict)
m.tab_inj = pyo.Param(m.j, m.tab_val, initialize=inj_dict_days)
m.tab_wit = pyo.Param(m.k, m.tab_val, initialize=wit_dict_days)
m.bsd_tab_wit = pyo.Param(m.k, m.tab_val, initialize=bsd_wit_dict_days)
m.state_to_date = pyo.Param(m.months, initialize=state_to_date)
m.bsd_wr = pyo.Param(m.i, initialize=bsd_wr_days)
m.bsd_wgv = pyo.Param(m.i, initialize=bsd_wgv_days)
m.mnd_wr = pyo.Param(m.i, initialize=mnd_wr)


# -----------------------------------------------------------
# Variables

m.x = pyo.Var(m.i, domain=pyo.NonNegativeIntegers, initialize=0, name='x')
m.y = pyo.Var(m.i, domain=pyo.NonNegativeIntegers, initialize=0, name='y')

m.t_inj = pyo.Var(m.i, m.j, domain=pyo.Binary, initialize=0, name='t_inj')
m.l_inj = pyo.Var(m.i, m.j, domain=pyo.Binary, initialize=0, name='l_inj')
m.u_inj = pyo.Var(m.i, m.j, domain=pyo.Binary, initialize=0, name='u_inj')

m.t_wit = pyo.Var(m.i, m.k, domain=pyo.Binary, initialize=0, name='t_wit')
m.l_wit = pyo.Var(m.i, m.k, domain=pyo.Binary, initialize=0, name='l_wit')
m.u_wit = pyo.Var(m.i, m.k, domain=pyo.Binary, initialize=0, name='u_wit')

m.z = pyo.Var(m.i, domain=pyo.NonNegativeIntegers, initialize=0, name='z')


# -----------------------------------------------------------
# Objective

m.objective = pyo.Objective(expr=(sum(m.y[i]*m.p[i] for i in m.i)-sum(m.x[i]*m.p[i] for i in m.i)),sense=pyo.maximize)


# -----------------------------------------------------------
# Constraints

m.constr_balance = pyo.Constraint(expr = sum(m.y[i] for i in m.i) <= z0 + sum(m.x[i] for i in m.i))

m.constr_terminal = pyo.Constraint(expr = m.z[date_end] == 0)
#m.constr_initial = pyo.Constraint(expr = m.z[date_start] == z0)

if empty_storage and empty_on >= date_start:
    m.constr_empty_storage = pyo.Constraint(expr = m.z[empty_on] == 0)

m.constr_capacity = pyo.ConstraintList()
for i in m.i:
    m.constr_capacity.add(m.z[i] <= m.wgv[i])

m.constr_gs = pyo.ConstraintList()
for i in m.i:
    if i == date_start:
        m.constr_gs.add(m.z[i] == z0 + m.x[i] - m.y[i])
        continue
    m.constr_gs.add(m.z[i] == m.z[i-delta] + m.x[i] - m.y[i])

m.constr_season = pyo.ConstraintList()
for i in m.i:
    if i.month >= season_inj[0] and i.month <= season_inj[1]:
        m.constr_season.add(m.y[i] == 0)
    else:
        m.constr_season.add(m.x[i] == 0)

m.constr_state_to_date = pyo.ConstraintList()
for i in m.i:
    for p in m.months:
        if i.month == p and i.day == 1:
            m.constr_state_to_date.add(m.z[i] >= m.state_to_date[p]*m.wgv[i])

m.constr_inj_low = pyo.ConstraintList()
for i in m.i:
    for j in m.j:
        m.constr_inj_low.add(m.tab_inj[(j,'lower')]*m.wgv[i] <= m.z[i] + m.m_const[i]*(1-m.l_inj[i,j]))
        m.constr_inj_low.add(m.tab_inj[(j,'lower')]*m.wgv[i] >= m.z[i] - m.m_const[i]*m.l_inj[i,j])
m.constr_inj_upp = pyo.ConstraintList()
for i in m.i:
    for j in m.j:
        m.constr_inj_upp.add(m.tab_inj[(j,'upper')]*m.wgv[i] >= m.z[i] - m.m_const[i]*(1-m.u_inj[i,j]))
        m.constr_inj_upp.add(m.tab_inj[(j,'upper')]*m.wgv[i] <= m.z[i] + m.m_const[i]*m.u_inj[i,j])
m.constr_inj_t = pyo.ConstraintList()
for i in m.i:
    m.constr_inj_t.add(sum(m.t_inj[i,j] for j in m.j) == 1)
    for j in m.j:
        m.constr_inj_t.add(m.u_inj[i,j] + m.l_inj[i,j] - 2*m.t_inj[i,j] >= 0)
        m.constr_inj_t.add(m.u_inj[i,j] + m.l_inj[i,j] - 2*m.t_inj[i,j] <= 1)
m.constr_inj = pyo.ConstraintList()
for i in m.i:
    m.constr_inj.add(m.x[i] <= m.ir[i]*sum(m.tab_inj[(j,'portion')]*m.t_inj[i,j] for j in m.j))

m.constr_wit_low = pyo.ConstraintList()
for i in m.i:
    for k in m.k:
        m.constr_wit_low.add(m.tab_wit[(k,'lower')]*m.wgv[i] <= m.z[i] + m.m_const[i]*(1-m.l_wit[i,k]))
        m.constr_wit_low.add(m.tab_wit[(k,'lower')]*m.wgv[i] >= m.z[i] - m.m_const[i]*m.l_wit[i,k])
m.constr_wit_upp = pyo.ConstraintList()
for i in m.i:
    for k in m.k:
        m.constr_wit_upp.add(m.tab_wit[(k,'upper')]*m.wgv[i] >= m.z[i] - m.m_const[i]*(1-m.u_wit[i,k]))
        m.constr_wit_upp.add(m.tab_wit[(k,'upper')]*m.wgv[i] <= m.z[i] + m.m_const[i]*m.u_wit[i,k])
m.constr_wit_t = pyo.ConstraintList()
for i in m.i:
    m.constr_wit_t.add(sum(m.t_wit[i,k] for k in m.k) == 1)
    for k in m.k:
        m.constr_wit_t.add(m.u_wit[i,k] + m.l_wit[i,k] - 2*m.t_wit[i,k] >= 0)
        m.constr_wit_t.add(m.u_wit[i,k] + m.l_wit[i,k] - 2*m.t_wit[i,k] <= 1)
m.constr_wit = pyo.ConstraintList()
for i in m.i:
    m.constr_wit.add(m.y[i] <= m.wr[i]*sum(m.tab_wit[(k,'portion')]*m.t_wit[i,k] for k in m.k))

# ---------------
# BSD

if check_bsd:
    m.constr_bsd_wr = pyo.ConstraintList()
    for i in m.i:
        m.constr_bsd_wr.add(0.3*m.bsd_wr[i] <= (m.wr[i]+m.mnd_wr[i])*sum(m.bsd_tab_wit[(k,'portion')]*m.t_wit[i,k] for k in m.k))

    m.constr_bsd_wgv = pyo.ConstraintList()
    for i in m.i:
        if i.day == 1:
            iter_day = i
            mon = []
            while iter_day.month == i.month:
                mon.append(iter_day)
                iter_day += delta
            m.constr_bsd_wgv.add(0.3*m.bsd_wgv[i] <= sum((m.wr[j]+m.mnd_wr[i])*sum(m.bsd_tab_wit[(k,'portion')]*m.t_wit[j,k] for k in m.k) for j in mon))
            if i == date_start:
                continue
            else:
                m.constr_bsd_wgv.add(0.3*m.bsd_wgv[i] <= m.z[i-delta])            

    # return m

Solution

In [14]:
#m1 = model()
m1=m

cplex_solver = SolverFactory('cplex', executable='/home/juzun/cplex/CPLEX_Studio/cplex/bin/x86-64_linux/cplex')
#cplex_solver.options["threads"] = 12
results = cplex_solver.solve(m1)

# scip_solver = SolverFactory('scip', executable='/home/juzun/scip/SCIPOptSuite-8.0.4-Linux/bin/scip')
# scip_solver.options = {
#     'limits/time': 300, 'limits/gap': 0.01, 'lp/threads': 12, 'parallel/minnthreads': 8}
# results = scip_solver.solve(m1, tee=True,) # load_solutions=False

print('Status: ', results.solver.status)
print('Termination condition: ', results.solver.termination_condition)
print(f'\nobj = {m1.objective()}') #pyo.value(m.objective)

Status:  ok
Termination condition:  optimal

obj = 125984659.81499961


In [22]:
m2 = model()

# my_solver = appsi.solvers.Cplex()

my_solver = appsi.solvers.Highs()
my_solver.highs_options = {'mip_rel_gap': 0.001} # 'threads': 12,

my_solver.config.time_limit = 300
my_solver.config.load_solution = False
my_solver.config.stream_solver = False

results = my_solver.solve(m2)

if (results.termination_condition == appsi.base.TerminationCondition.optimal) or (results.best_feasible_objective is not None):
    print('\nTermination condition: ', results.termination_condition)
    results.solution_loader.load_vars()
print(f'\nobj = {m2.objective()}, {results.best_feasible_objective}')

Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms

Termination condition:  TerminationCondition.maxTimeLimit

obj = 125959479.30899976, 125959479.309


NEOS

In [6]:
# os.environ['NEOS_EMAIL'] = 'zapletalja@gmail.com'
# manager = pyo.SolverManagerFactory('neos')
# m = model(date_start,z0,wgv1,ir1,wr1,date_change,wgv2,ir2,wr2,date_end,season_inj,state_to_date,delta,dates,prices_daily,empty_storage,empty_on,inj_dict,wit_dict,bsd_wr,bsd_wgv)
# sol = manager.solve(m, solver = 'cplex')
    
# print('Status: ', sol.solver.status)
# print('Termination condition: ', sol.solver.termination_condition)
# print(f'\nobj = {m.objective()}')

Extract data from model

In [26]:
m = m1

In [8]:
x_dict = m.x.extract_values()
y_dict = m.y.extract_values()
z_dict = m.z.extract_values()
t_inj = m.t_inj.extract_values()
l_inj = m.l_inj.extract_values()
u_inj = m.u_inj.extract_values()
t_wit = m.t_wit.extract_values()
l_wit = m.l_wit.extract_values()
u_wit = m.u_wit.extract_values()
ir_dict = m.ir.extract_values()
wr_dict = m.wr.extract_values()
bsd_wr_dict = m.bsd_wr.extract_values()
bsd_wgv_dict = m.bsd_wgv.extract_values()
wgv = m.wgv.extract_values()
wr = m.wr.extract_values()

op_dict = {key: x_dict[key]-y_dict[key] for key in dates}

index = dates
x = np.array(list(x_dict.values()))
y = -np.array(list(y_dict.values()))
z = np.array(list(z_dict.values()))
bsd_wr = -0.3*np.array(list(bsd_wr_dict.values()))
bsd_wgv = 0.3*np.array(list(bsd_wgv_dict.values()))
op = np.array(list(op_dict.values()))
prices = prices_daily['price'].to_list()

tab_inj = m.tab_inj.extract_values()
tab_wit = m.tab_wit.extract_values()
max_op = np.empty(x.size)
for i in range(max_op.size):
    if index[i].month >= season_inj[0] and index[i].month <= season_inj[1]:
        max_op[i] = ir_dict[index[i]]*sum(tab_inj[(j,'portion')]*t_inj[index[i],j] for j in list(m.j))
    else:
        max_op[i] = -wr_dict[index[i]]*sum(tab_wit[(k,'portion')]*t_wit[index[i],k] for k in list(m.k))
max_op_dict = {dates[i]: max_op[i] for i in range(len(dates))}

Graphs

In [10]:
fig = po.Figure()
fig.add_trace(po.Scatter(x=index, y=max_op, name='Maximum', line_color='#ffa600', mode='lines'))
fig.add_trace(po.Scatter(x=index, y=bsd_wr, name='BSD WR', line_color='#bac52d', mode='lines'))
fig.add_trace(po.Scatter(x=index, y=op, name='Operations', fill='tozeroy', line_color='#74d576', mode='lines'))
fig.add_trace(po.Scatter(x=index, y=z, name='WGV', fill='tozeroy', line_color='#34dbeb', yaxis = 'y2'))
fig.update_layout(
    title = 'Gas storage optimization',
    xaxis_title = 'Date',
    yaxis = dict(
        title = 'Operations [MWh/day]',
        range = [-35000,35000]),
    yaxis2 = dict(
        title = "WGV [MWh]",
        side = 'right',
        overlaying = 'y',
        titlefont = dict(color='#34dbeb'),
        tickfont = dict(color='#34dbeb')),
    legend = dict(
        orientation = "v",
		x = 1.06,
        xanchor = 'left',
		y = 1
    )
)
fig.update_xaxes(fixedrange=False)
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='grey')
fig.show()

In [29]:
fig = po.Figure()
fig.add_trace(po.Scatter(x=index, y=mnd_state_ar, name='MND WGV', fill='tozeroy', line_color='#34dbeb'))
fig.add_trace(po.Scatter(x=index, y=mnd_wr_ar, name='MND operations', fill='tozeroy', line_color='#74d576', mode='lines'))
fig.update_layout(
    title = 'Gas storage MND'
)
fig.update_xaxes(fixedrange=False)
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='grey')
fig.show()

Export to pdf

In [30]:
mnd_z_state = {}
for key in dates:
    mnd_z_state[key] = z_dict[key] + mnd_state[key]
mnd_z_state_ar = np.array(list(mnd_z_state.values()))

total_wr = {key: op_dict[key]-mnd_wr[key] for key in dates}
total_wr_ar = np.array(list(total_wr.values()))
max_mnd = {key: (-mnd_max_wr if mnd_state[key]>0 else 0) for key in dates}
max_mnd_ar = np.array(list(max_mnd.values()))
z_rel = {key: z_dict[key]/wgv[key] for key in z_dict}
z_rel_ar = np.array(list(z_rel.values()))
max_bsd = {key: 
           (wr[key]+mnd_max_wr if z_rel[key] > 0.35 else abs(max_op_dict[key]+max_mnd[key])) if key<=dt.date(2024,3,31) else 0
           for key in dates}
max_bsd_ar = np.array(list(max_bsd.values()))
z_monthly = {key: z0+mnd_wgv_cap if key==date_start else (mnd_z_state[key-delta] if key.day==1 else 0) for key in dates}
bsd_z_monthly = {key: z_monthly[key] if key<=dt.date(2024,3,31) else 0 for key in dates}
bsd_z_monthly_ar = np.array(list(bsd_z_monthly.values()))

In [31]:
daily_export = pd.DataFrame(
    list(zip(
        list(prices_daily['year']),list(prices_daily['month']),total_wr_ar,z,z_rel_ar,mnd_state_ar,mnd_z_state_ar,
        max_op,max_mnd_ar,max_bsd_ar,bsd_z_monthly_ar)),
    index=index,
    columns=['Rok','M','W/I','Stav RWE','Stav RWE %','Stav MND','Stav Total','Max C RWE','Max C MND','Max C BSD','Volume GS BSD'])

daily_export_agg = daily_export.groupby(['Rok','M']).agg(
    w_i=('W/I','sum'), bsd_vykon=('Max C BSD','min'),
    bsd_objem_cap=('Volume GS BSD','sum'), bsd__objem_max_w=('Max C BSD','sum')
)
stav_gs = []
stav_gs.append(z0+mnd_wgv_cap)
for i in range(1,len(daily_export_agg.w_i.values)):
    stav_gs.append(stav_gs[i-1]+daily_export_agg.w_i.values[i])

monthly_export = pd.DataFrame(    
    list(zip(
        prices_monthly['year'].values,prices_monthly['month'].values,
        daily_export_agg.w_i.values, stav_gs, daily_export_agg.bsd_vykon.values,
        daily_export_agg.bsd_objem_cap.values, daily_export_agg.bsd__objem_max_w.values,
    )),
    columns=['Rok','M','W/I','Stav GS','BSD Výkon','BSD Objem (cap)','BSD Objem (max W)']
)
monthly_export['BSD Objem'] = monthly_export[['BSD Objem (cap)','BSD Objem (max W)']].T.min()

name = 'export.xlsx'
if os.path.exists(name):
    with pd.ExcelWriter(name, mode='a', engine='openpyxl', if_sheet_exists='replace') as w:
        daily_export.to_excel(w, sheet_name='data_daily', index=True, index_label='Datum')
        monthly_export.to_excel(w, sheet_name='data_monthly', index=False)
else:
    with pd.ExcelWriter(name, mode='w', engine='openpyxl') as w:
        daily_export.to_excel(w, sheet_name='data_daily', index=True, index_label='Datum')
        monthly_export.to_excel(w, sheet_name='data_monthly', index=False)

In [34]:
monthly_export

,Rok,M,W/I,Stav GS,BSD Výkon,BSD Objem (cap),BSD Objem (max W),BSD Objem
0,2023,11,0.000000e+00,2.133317e+06,32780.00,2133317.0,983400.00,983400.00
1,2023,12,-1.082000e+03,2.132235e+06,32780.00,2133317.0,1016180.00,1016180.00
2,2024,1,-5.076340e+05,1.624601e+06,32780.00,2132235.0,1016180.00,1016180.00
3,2024,2,-9.177780e+05,7.068230e+05,24853.14,1624601.0,942693.14,942693.14
4,2024,3,-6.444730e+05,6.235000e+04,16887.39,706823.0,650961.09,650961.09
5,2024,4,1.255600e+04,7.490600e+04,0.00,0.0,0.00,0.00
6,2024,5,4.829920e+05,5.578980e+05,0.00,0.0,0.00,0.00
7,2024,6,5.351930e+05,1.093091e+06,0.00,0.0,0.00,0.00
8,2024,7,4.428070e+05,1.535898e+06,0.00,0.0,0.00,0.00
9,2024,8,1.479820e+05,1.683880e+06,0.00,0.0,0.00,0.00


In [33]:
daily_export

,Rok,M,W/I,Stav RWE,Stav RWE %,Stav MND,Stav Total,Max C RWE,Max C MND,Max C BSD,Volume GS BSD
2023-11-01,2023,11,0.0,2078317.0,1.0,55000,2133317.0,-12745.20,-917,32780.0,2133317.0
2023-11-02,2023,11,0.0,2078317.0,1.0,55000,2133317.0,-12745.20,-917,32780.0,0.0
2023-11-03,2023,11,0.0,2078317.0,1.0,55000,2133317.0,-12745.20,-917,32780.0,0.0
2023-11-04,2023,11,0.0,2078317.0,1.0,55000,2133317.0,-12745.20,-917,32780.0,0.0
2023-11-05,2023,11,0.0,2078317.0,1.0,55000,2133317.0,-12745.20,-917,32780.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-03-27,2025,3,0.0,0.0,0.0,0,0.0,-4533.12,0,0.0,0.0
2025-03-28,2025,3,0.0,0.0,0.0,0,0.0,-4533.12,0,0.0,0.0
2025-03-29,2025,3,0.0,0.0,0.0,0,0.0,-4533.12,0,0.0,0.0
2025-03-30,2025,3,0.0,0.0,0.0,0,0.0,-4533.12,0,0.0,0.0
